In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import datetime
import os
import s3fs
import boto3
from io import StringIO

def fetch_spotify_data():
    #Connect to the API
    
    client_id = "XXXXXXXXX" 
    client_secret = "XXXXXXX"

    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    #Retrieve IDs for each track
    def getTrackIDs(user, playlist_id):
        ids = []
        playlist = sp.user_playlist(user, playlist_id)
        for item in playlist['tracks']['items']:
            track = item['track']
            ids.append(track['id'])
        return ids

    ids = getTrackIDs('Rene', '7fvHqPdOTcsI63laE4mtBW')

    #Check we are getting correct count of songs
    print(len(ids))
    print(ids)

    #Create a function used to grab all track info from IDs
    def getTrackFeatures(id):
      meta = sp.track(id)
      features = sp.audio_features(id)

      # meta
      name = meta['name']
      album = meta['album']['name']
      artist = meta['album']['artists'][0]['name']
      release_date = meta['album']['release_date']
      length = meta['duration_ms']
      popularity = meta['popularity']

      # features
      acousticness = features[0]['acousticness']
      danceability = features[0]['danceability']
      energy = features[0]['energy']
      instrumentalness = features[0]['instrumentalness']
      liveness = features[0]['liveness']
      loudness = features[0]['loudness']
      speechiness = features[0]['speechiness']
      tempo = features[0]['tempo']
      time_signature = features[0]['time_signature']

      track = [name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
      return track


    tracks = []
    for i in range(len(ids)):
        track = getTrackFeatures(ids[i])
        print(track)  # Check the printed output for this
        tracks.append(track)


    # create dataframe
    df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
    
    # Create an S3 Client
    s3 = boto3.client('s3', aws_access_key_id="XXXXXXXXX", aws_secret_access_key="XXXXXXXXX")
    
    # Convert DataFrame to CSV
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)
    
    # Add timestamp to file name
    key = "Spotify_data.csv"
    current_timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    base_name = key.split('.')[0]
    extension = key.split('.')[-1]
    new_key = f"{base_name}_{current_timestamp}.{extension}"

    # Upload CSV to S3 with the new timestamped filename
    s3.put_object(Bucket="rene-test-project-bucket", Key=new_key, Body=csv_buffer.getvalue())


    
    print(df)

# To run the function:
fetch_spotify_data()
